### **[01] 비동기 통신 이해하기**

##### **01. 백엔드 : ``main.py`` (파이썬/FastAPI)**

이 파일은 식당으로 치면 **주방**으로 손님(FrontEnd)의 주문을 기다렸다가 음식을 만들어 보내주는 역할

In [ ]:
from fastapi import FastAPI # FastAPI라는 요리 도구 세트를 가져옴
from fastapi.middleware.cors import CORSMiddleware # 다른 포트에서 오는 요청을 허용해주는 '통행증' 관리 도구를 가져옴

# 이 식당에 들어올 수 있는 손님(프론트엔드 주소) 명단
origins = [
    "http://localhost:5173" # 리액트가 5173 포트를 쓰고 있다면 이 주소를 허용함
]

app = FastAPI() # FastAPI 앱 객체를 생성 (!!식당 오픈!!)

# 보안 정책(CORS) 설정을 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins, # 위에서 정한 주소(리액트)만 접속 허락
    allow_credentials=True, # 쿠키나 로그인 정보 등을 주고받는 것을 허락
    allow_methods=["*"], # get, post 등 모든 방식의 요청을 허락
    allow_headers=["*"], # 모든 종류의 메타 정보(헤더)를 주고받는 것을 허락
)

# 손님이 "주소/" 경로로 get 요청(단순 조회)을 보내면 실행되는 함수
@app.get("/")
def read_root() :
    # 손님에게 줄 데이터(메뉴)를 딕셔너리 형태로 반환
    return {"status" : True, "result" : ["공유는 해드림"]}

##### **2. 프론트엔드 : ``Home.jsx`` (리액트)**

이 파일은 **손님** 역할을 하며, 화면에 버튼을 보여주고, 버튼을 누르면 주방(FastAPI)에 음식을 주문함

In [ ]:
import axios from 'axios' // 서버와 통신하기 위한 심부름꾼 도구인 'axios'를 가져옴 (종업원)

const Home = () => {
  // 버튼을 눌렀을 때 실행될 이벤트 함수를 정의
  const btn1Event = () => {
    console.log("btn1 호출") // 브라우저 콘솔에 로그를 남김

    // 1. axios를 이용해 백엔드 서버(8000번 포트)에 데이터를 달라고 요청
    axios.get("http://localhost:8000")
    // 2. 서버가 성공적으로 응답을 보내주면 실행됨 (res는 서버에서 온 결과 봉투)
    .then(res => {
      console.log(res.data) // 서버에서 보내준 진짜 데이터 ({status : True...})를 출력
      
      // 서버 응답의 status가 True라면 알림창에 첫 번째 결과값을 띄움
      if(res.data.status) {
        alert(res.data.result[0]) // "공유는 해드림"이 뜸
      } else {
        alert("오류 발생") // status가 False라면 오류 알림을 띄움
      }
    })

    // 3. 서버가 꺼져있거나 주소가 틀리는 등 '통신 에러'가 나면 실행됨
    .catch(err => console.error(err))

    // 4. 성공하든 실패하든 통신이 끝나면 무조건 마지막에 실행됨
    .finally(() => console.log("완료"))
  }

  // 화면에 보여줄 HTML 구조를 반환
  return (
    <div className="text-center">
      <h1>메인 화면입니다.</h1>
      <button type="button" onClick={btn1Event}>FastAPI 확인</button>
    </div>
  )
}

export default Home // 다른 파일에서 이 컴포넌트를 가져다 쓸 수 있게 내보냄

##### **3. 핵심 포인트**

1. **CORS**  
브라우저는 기본적으로 **'나(5173)'**와 **'남(8000)'**의 통신을 막기 때문에 백엔드에서 ``CORSMiddleware``로 "얘는 내 친구야"라고 허락해줘야 함

2. **Axios**  
자바스크립트에서 서버와 대화하는 가장 대중적인 방법.  
``then``은 성공, ``catch``는 실패를 담당

3. **포트(Port)**  
리액트(5173)와 FastAPI(8000)는 각자의 방 번호가 다르기 때문에 이 번호를 정확히 맞춰서 요청해야 함

### **[02] 로그인 / 로그아웃 이해하기** 

**【 분석? 이렇게 해보자 】**  
1. **Backend(``main.py``)**  
서버가 어떤 메뉴(API)를 준비하고 있는지, 데이터의 형식을 어떻게 정의했는지 먼저 파악
  
2. **Hooks(``AuthContext.js`` 등)**  
여러 화면에서 공통으로 사용할 "데이터 저장소"와 "관리 규칙"을 이해하기  
  
3. **Frontend(``Login.jsx``, ``Home.jsx``)**  
사용자가 보는 화면에서 어떻게 서버로 주문을 넣고, 받은 데이터를 화면에 그리는지 확인

##### **00. 동작원리**

**식당**에 비유해서 보기

- **Backend**  
요리를 하는 **주방**.  
정해진 메뉴(URL)로 주문이 들어오면 요리(데이터 처리)를 해서 내보냄

- **Frontend**  
손님이 앉는 **테이블**.  
메뉴판(UI)을 보고 주문(Request)을 하며, 나온 음식(Response)을 맛있게 보여줌

- **Hooks (Context)**  
손님이 식당에 들어올 때 받는 **번호표나 예약 정보**와 같음  
로그인을 했는지 안 했는지 식당 전체가 공유하는 정보를 관리

##### **01. 백엔드 : ``main.py``**

- **Point**  
데이터의 흐름과 쿠키(Cookie) 개념 이해하기  
단순히 데이터를 주고받는 것을 넘어, "이 사용자가 누구인지" 기억하는 시스템의 기초가 담겨있음

**1. 도구 가져오기 및 환경 설정**

In [ ]:
# -- 1. 도구 가져오기 및 환경 설정 --
from fastapi import FastAPI, Request, Response # FastAPI의 핵심 기능(요청, 응답 처리)을 가져옴
from fastapi.middleware.cors import CORSMiddleware # 다른 포트(리액트)의 접속을 허용하는 '통행증' 도구
from pydantic import BaseModel # 데이터의 규격(형식)을 정해주는 도구

# 허용할 프론트엔드 주소 (리액트 기본 포트 5173)
origins = [
    "http://localhost:5173"
]

# [로그인 규격] 손님이 보낼 데이터는 반드시 이 형식(이메일, 비밀번호)이어야 한다고 정함
class LoginModel(BaseModel) :
    email : str # 문자열 형태의 이메일
    pwd : str # 문자열 형태의 비밀번호

app = FastAPI() # FastAPI 주방을 오픈

# CORS 설정 : 리액트 앱이 이 서버에 안전하게 접근할 수 있도록 문을 열어줌
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True, # 【중요】리액트 앱이 이 서버에 안전하게 접근할 수 있도록 문을 열어줌 
    allow_methods=["*"], # 모든 방식(get, post 등) 허용
    allow_headers=["*"], # 모든 메타 정보 허용
)

**2. 기능(메뉴) 만들기**

**(1) 기본 화면 (GET /)**

In [ ]:
@app.get("/")
def read_root() :
    # 서버가 잘 작동하는지 확인하는 용도의 간단한 응답
    return {"status" : True, "result" : ["공유는 해드림"]}

**(2) 로그인 (POST /login)** → 현재 코드의 핵심 부분

In [ ]:
@app.post("/login")
def login(loginModel : LoginModel, response : Response) :
    # response.set_cookie : 손님(브라우저)의 가방에 'user'라는 이름의 이름표(쿠키)를 넣어줌
    response.set_cookie(
        key = "user",               # 이름표의 이름
        value = loginModel.email,   # 이름표에 적힐 내용 (사용자 이메일)
        max_age = 60 * 60,          # 1시간 동안 유지 (60초 * 60분)
        expires = 60 * 60,          # 만료 시간 설정
        path = "/",                 # 모든 페이지에서 이 쿠키를 사용할 수 있음
        domain = "localhost",       # 이 주소에서만 쿠키가 유효함
        secure = True,              # 보안 연결(HTTPS)에서만 전송 (로컬 테스트 시 간혹 주의)
        httponly = True,            # 자바스크립트가 쿠키를 훔쳐가지 못하게 방어
        samesite = "lax",           # 서로 다른 사이트 간의 요청 시 보안 설정
    )
    # 로그인 성공 메시지와 함께 받은 데이터를 그대로 돌려줌
    return {"status" : True, "model" : loginModel}

**(3) 로그아웃 (POST /logout)**

In [ ]:
@app.post("/logout")
def logout(response : Response) :
    # 손님의 가방에서 'user' 이름표를 찾아 삭제
    response.delete_cookie(key="user")
    return {"staus" : True}

**(4) 사용자 확인 (GET /user)**

In [ ]:
@app.get("/user")
def user(request : Request) :
    # 손님이 가져온 가방에서 'user'라는 이름표가 있는지 확인
    email = request.cookies.get("user")
    if email :
        # 이름표가 있다면 "너는 누구구나!" 하고 알려줌
        return {"status" : True, "me" : email}
    else :
        # 이름표가 없다면 로그인하지 않은 상태로 간주
        return {"status" : False}

**(5) Request(요청)과 Response(응답)**

**<span style="background-color:yellow">핵심 개념</span>**  
**<span style="color:red">손님의 편지 vs 주방장의 봉투</span>**

웹 통신은 항상 **[손님의 요청 → 주방장의 처리 → 주방장의 응답]** 순서로 일어남

- **``Request`` (요청)**  
손님(브라우저)이 주방장(서버)에게 보내는 **편지**임  
"나 이런 데이터(로그인 정보) 가지고 있고, 내 가방엔 이런 이름표(쿠키)가 있어!" 라는 정보가 담김

##### **02. Hook : ``AuthProvider.js`` 등**

**【``js`` 파일과 ``jsx`` 파일 두 가지를 쓰는 이유】**  

보통 리액트 숙련자들은 **데이터 저장소의 정의**와 **데이터를 관리하는 로직**을 분리하고 싶어 함

- ``AuthProvider.js`` **(정의서)**  
"우리는 ``AuthContext``라는 이름의 데이터 보관함을 만들 거야!" 라고 선언만 하는 파일.  
HTML 요소(JSX)가 없으므로 ``.js`` 확장자를 씀

- ``AuthProvider.jsx`` **(실행부)**  
"그 보관함에 ``isLogin`` 상태도 넣고, 로그인 / 로그아웃 함수도 만들어서 자식 컴포넌트들(``children``)에게 뿌려줄게!" 라고 실제 로직을 짜는 파일.  
HTML 형태의 태그가 포함되므로 ``.jsx`` 확장자를 씀

**<sapn style="background-color:yellow">핵심 포인트</span>**  
  
``AuthProvider.jsx`` 안에도 ``export const AuthContext = createContext()``가 이미 들어가 있어서 ``js`` 파일은 삭제하고 ``jsx`` 하나만 사용해도 코드는 똑같이 작동함

**(1) AuthProvider.js**

이 파일은 실제 로직을 담기보다, 데이터가 담길 **바구니**를 정의함

In [ ]:
import { createContext } from 'react'; // 리액트 패키지에서 Context(데이터 바구니)를 만드는 도구를 가져옴

// 외부에서 사용할 수 있도록 'AuthContext'라는 이름의 데이터 바구니를 만들어 내보냄
// createContext(null)은 처음 바구니를 만들 때 안에 아무것도(null) 넣지 않겠다는 뜻

export const AuthContext = createContext(null);


**(2) AuthProvider.jsx**

이 파일은 앱 전체의 **로그인 상태 관제탑**이라 할 수 있음

**1. ``import`` 영역 : 필요한 도구들 가져오기**

In [ ]:
import { createContext, useContext, useState, useEffect } from "react"
// React의 핵심 기능을 가져옴
// - createContext : 공용 데이터 바구니를 만들기
// - useContext : 바구니에서 데이터를 꺼내 씀
// - useState : 변하는 데이터(상태)를 관리
// - useEffect : 앱이 시작되거나 특정 조건일 때 실행할 코드를 적음

import { useNavigate } from "react-router";
// 페이지를 강제로 이동시키는 도구 (예 : 로그인 후 메인으로 보내기)

import axios from "axios"
// 서버(FastAPI)와 대화하기 위한 심부름꾼

**2. ``api`` 기본 설정 영역(Axios 세팅) : 서버와의 약속**

In [ ]:
const api = axios.create({
  // .env 파일에서 서버 주소를 가져오고, 없으면 기본 8000번을 씀
  baseURL: import.meta.env.VITE_APP_FASTAPI_URL || "http://localhost:8000",
  
  // 【중요】백엔드가 보낸 쿠키를 브라우저에 저장하고 다시 보낼 수 있게 허용함
  withCredentials: true,
  
  headers: {
    "Content-Type": "application/json", // 우리는 데이터를 JSON 형식으로 주고받겠다
  },
})

**3. AuthProvider 구현 영역 : 상태(State)와 로그인 함수 → 실제 영역**

In [ ]:
// (1) 공용 바구니(AuthContext)를 만들기
export const AuthContext = createContext()

const AuthProvider = ({children}) => {
  // (2) 로그인 여부를 기억할 변수(isLogin)를 만듦, 초기값은 false(로그아웃 상태)
  const [isLogin, setIsLogin] = useState(false)
  const navigate = useNavigate()

  // (3) 로그인 성공 시 실행할 함수
  const setAuth = status => {
    localStorage.setItem("user", status) // 브라우저 메모리에 '나 로그인했음'이라고 기록
    setIsLogin(status) // 리액트 화면을 로그인 상태로 바꿈
    navigate("/") // 메인 페이지로 보냄
  }

  // (4) 로그아웃 함수
  const removeAuth = () => {
    api.post("/logout") // 서버에 "나 이제 갈게" 라고 알림
    .then(res => {
      console.log(res)
      localStorage.removeItem("user") // 브라우저 메모리에서 기록을 지움
      setIsLogin(false) // 리액트 화면을 로그아웃 상태로 바꿈
      navigate("/") // 메인 페이지로 보냄
    })
    .catch(err => console.error(err))
  }

  // (5) 새로고침 방어 로직
  useEffect(()=> {
    // 앱이 처음 켜질 때 브라우저 메모리에 'user' 기록이 남아있다면,
    // 새로고침을 했어도 로그인 상태(true)로 인정해줌
    if(localStorage.getItem("user")) setIsLogin(true)
  }, [])

  return (
    // (6) 이 바구니(Provider)에 데이터와 함수들을 담아서 {children}(자식 컴포넌트들)에게 뿌려줌
    <AuthContext.Provider value={{ isLogin, setAuth, removeAuth }}>
      {children}
    </AuthContext.Provider>
  )
}

// (7) 다른 파일에서 이 바구니의 데이터를 쉽게 꺼내 쓸 수 있게 만든 수신기(useAuth)
export const useAuth = () => useContext(AuthContext)

export default AuthProvider

**<span style="background-color: yellow">동작원리</span>**

이 코드는 **전역 방송국** 이라고 생각해보자

1. **방송국 설립 (``createContext``)**  
정보를 전달할 채널을 개설  
  
2. **방송 송출 (``Provider``)**  
``isLogin`` 정보와 로그인/로그아웃 함수를 공중에 뿌림  
  
3. **수신기 (``useAuth``)**  
`Login.jsx`나 ``Home.jsx``같은 개별 페이지에서 안테나를 세워 방송 내용을 실시간으로 받아옴  
  
4. **메모리 (localStorage)**  
브라우저라는 일기장에 로그인 여부를 적어두어, 전원이 꺼졌다 켜져도(새로고침) 누구인지 잊지 않게 함

**<span style="background-color: yellow">포인트</span>**

- **``childeren``**  
이 ``AuthProvider``가 감싸고 있는 모든 컴포넌트들을 말함.  
보통 ``App.jsx``에서 전체를 감싸서 앱 어디서든 로그인을 알 수 있게 함  
  
- **``withCredentials``**  
FastAPI 서버에서 설정한 **쿠키**를 주고받으려면 프론트엔드에서도 반드시 이 옵션을 켜줘야 함